In [4]:
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup

In [6]:
url = 'https://www.cvk.gov.ua/pls/vp2019/wp300pt001f01=719.html'
tables = pd.read_html(url, encoding='cp1251')
final_results_df = tables[1]
final_results_df.drop('Графік', axis=1, inplace=True)
final_results_df

f:\programs\anaconda3\envs\cv-project\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,Кандидат,"% голосів виборців, поданих за кандидата","Кількість голосів виборців, поданих за кандидата"
0,Зеленський Володимир,30.24,5 714 034
1,Порошенко Петро,15.95,3 014 609
2,Тимошенко Юлія,13.40,2 532 452
3,Бойко Юрій,11.67,2 206 216
4,Гриценко Анатолій,6.91,1 306 450
5,Смешко Ігор,6.04,1 141 332
6,Ляшко Олег,5.48,1 036 003
7,Вілкул Олександр,4.15,784 274
8,Кошулинський Руслан,1.62,307 244
9,Тимошенко Юрій,0.62,117 693


In [ ]:
def create_folder(where, new_folder):
    if new_folder not in os.listdir(where):
        os.makedirs(f'{where}/{new_folder}')
        print(f'Folder {new_folder} was successfully created ({new_folder in os.listdir(where)})')


create_folder(where='../data/raw/', new_folder='2019')


def get_tables_to_csv(link, where, ind=1):
    tables = pd.read_html(link, encoding='cp1251')
    df = tables[ind]
    df.drop('Графік', axis=1, inplace=True)
    df.to_csv(where, index=False)

    return df


def get_soup(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup


def get_name(link):
    soup = get_soup(link=link)
    return '_'.join(soup.find_all('a', {'class': 'a2'})[0].text.strip().split(' '))


def get_file_hrefs(link, df):
    if 'Кандидат' in df.columns:
        col = 'Кандидат'
        existing_values = df[col].apply(lambda x: x.replace(' ', '')).values
    else:
        col = 'Регіон'
        existing_values = df[col].values
    soup = get_soup(link=link)
    return [name['href'] for name in soup.find_all('a', {'class': 'a1'}) 
            if name.text in existing_values]


def get_name_from_h2(link):
    soup = get_soup(link=link)
    return '_'.join(soup.find_all('h2', {'class': 'w100 cntr pdd15'})[0].text.strip().split(' '))

# first page (final results)
link = 'https://www.cvk.gov.ua/pls/vp2019/wp300pt001f01=719.html'
base_url = 'https://www.cvk.gov.ua/pls/vp2019/'

all_links = []

# second pages (regions)
all_links = get_file_hrefs(link=link, df=final_results_df)
all_links = [f'{base_url}{link}' for link in all_links]

for link in all_links:
    name = get_name(link)
    df = get_tables_to_csv(link=link, where=f'../data/raw/2019/{name}.csv')
    
    hrefs = get_file_hrefs(link=link, df=df)
    hrefs = [f'{base_url}{link}' for link in hrefs]

    # third pages (districts)
    for href in hrefs:
        create_folder('../data/raw/2019/', name)
        name_inner = get_name_from_h2(href)
        get_tables_to_csv(link=href, where=f'../data/raw/2019/{name}/{name_inner}.csv', ind=2)
        time.sleep(1)
    time.sleep(2)